# Import Libraries

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split

# adding new libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.layers import Activation 
from tensorflow.keras import layers, Input
from tensorflow.keras.regularizers import l1, l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler  # Import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from keras.optimizers import SGD
from tensorflow.keras.losses import MeanAbsoluteError
from keras.regularizers import l2


# Import Data

In [25]:
# df = pd.read_csv('../../res/model_data/batemo_model_data.csv')
# df = df.drop('SOH', axis=1) # Remove SOH from input data
# 
# # add moving averages of V, I, SOC, and T_surface to the input data
# df['V_ma'] = df['V'].rolling(window=10).mean()
# df['I_ma'] = df['I'].rolling(window=10).mean()
# df['T_surf_ma'] = df['T_surf'].rolling(window=10).mean()
# 
# # drop the first 10 rows of the data since the moving averages are NaN for the first 10 rows
# df = df.dropna()
# 
# df

# Import Data
df = pd.read_csv('../../res/model_data/batemo_model_data.csv')
df.describe()


,V,I,SOC,T_surf,SOH
count,1.640946e+06,1.640946e+06,1.640946e+06,1.640946e+06,1.640946e+06
mean,3.471013e+00,3.610140e+00,4.188879e+01,5.180549e+01,8.566437e+01
std,8.445745e-01,2.830966e+01,4.022232e+01,2.451698e+01,9.026790e+00
min,-9.394656e-01,-5.500295e+01,-5.753759e+00,2.499965e+01,7.000000e+01
25%,3.144859e+00,-1.900000e+01,1.230206e+00,2.583835e+01,7.800000e+01
50%,3.693897e+00,5.000000e+00,3.474387e+01,4.826449e+01,8.600000e+01
75%,4.064538e+00,2.600000e+01,8.456010e+01,7.304868e+01,9.400000e+01
max,4.723157e+00,5.500289e+01,1.070423e+02,1.015903e+02,1.000000e+02


# Train Test Split

In [26]:
# split data to be 80% training and 20% testing using sklearn
# train, test = train_test_split(df, test_size=0.2)
# 
# # define the input vectors and the corresponding labels 
# # SOC from the input data is our corresponding label Y and all other data is the input features X
# X_train = train.drop('SOC', axis=1)
# Y_train = train['SOC']
# X_test = test.drop('SOC', axis=1)
# Y_test = test['SOC']

# Define training variables
X = df[['V', 'I', 'T_surf']]
Y = df[['SOC', 'SOH']]

# Normalize input data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Reshape input data for LSTM
x_train = x_train.values.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.values.reshape((x_test.shape[0], x_test.shape[1], 1))

df

,V,I,SOC,T_surf,SOH
0,4.200000,0.000000,100.000000,25.000000,100
1,4.198758,-0.225199,100.000000,25.000000,100
2,4.197524,-0.449161,100.000000,25.000000,100
3,4.196296,-0.671867,100.000000,25.000000,100
4,4.190443,-1.733425,100.000000,25.000000,100
...,...,...,...,...,...
1640941,3.040704,-41.000000,14.330671,95.558472,70
1640942,3.019722,-41.000000,13.112293,96.489709,70
1640943,2.985720,-41.000000,11.355930,97.840139,70
1640944,2.945987,-41.000000,9.599568,99.207108,70


# Fit Model

In [30]:
# create an LSTM to train our model for finding SOC 
# define model
# model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(x_train.shape[1], 1)))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
# model.summary()
# model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), verbose=2)

# Define model
model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(50, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2]), kernel_regularizer=l2(0.01)))

model.add(Dense(2))  # Output layer for two continuous values: SOC and SOH
# model.compile(optimizer='adam', loss='mse')  # Use mean squared error for regression

# Compile model with Adam optimizer and custom learning rate
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mse')  

model.summary()
# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), verbose=2)

# TODO: THINGS TO CONSIDER FOR FINE-TUNING HYPERPARAMETERS
# TODO: 1) consider altering the depth of the network. currently we have 1 hidden layer w/ 50 nodes 
# TODO: 2) consider altering the width of the network. current there are 50 nodes, but that could be fine-tuned 
# TODO: 3) consider altering the activation function. right now it is set to be relu 
# TODO: 4) consider altering the optimizier. right now it is set to Adam
# TODO: 5) consider altering the number of epochs 

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 50)                10400     
                                                                 
 dense_9 (Dense)             (None, 2)                 102       
                                                                 
Total params: 10502 (41.02 KB)
Trainable params: 10502 (41.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
41024/41024 - 106s - loss: 35.7812 - val_loss: 16.9710 - 106s/epoch - 3ms/step
Epoch 2/10
41024/41024 - 101s - loss: 15.6516 - val_loss: 16.0964 - 101s/epoch - 2ms/step
Epoch 3/10
41024/41024 - 101s - loss: 14.0158 - val_loss: 13.0851 - 101s/epoch - 2ms/step
Epoch 4/10
41024/41024 - 102s - loss: 13.3788 - val_loss: 12.3884 - 102s/epoch - 2ms/step
Epoch 5/10
41024/41024 - 102s - loss: 12.9372 - val_lo

KeyboardInterrupt: 

# Test Model / Predict

In [23]:
# print accuracy
stats = model.evaluate(x_test, y_test, return_dict=True)

10256/10256 [==============================] - 18s 2ms/step - loss: 11.0832


In [39]:
# Print Accuracy and model statistics summary
print(f'Accuracy: {stats.get("accuracy")}')

Accuracy: 0.0010439339093863964


# Save Model and Export (Optimize for Size)